<a href="https://colab.research.google.com/github/davidfdezmartin/Chatbot-con-FastAPI-Streamlit-y-LangChain/blob/main/documentos_y_matcomplement/Colab%20de%20Pruebas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Aquí monto requeriments2.txt del GitHub pero com la libreria typer==0.09.0, que por un lado debe ser <0.10.00 (por eso he puesto 0.09.0 y por otro mayor que para fastapi-cli 0.0.4 requires typer>=0.12.3, a REVISARLO

Si se ha instalado longchain y falla, quitar el # del comentario de esta celda.
Uso rutas absolutas respecto al GitHub al principio para que no falle.

In [2]:
#  %pip uninstall -y langchain langchain-community
%pip install -r "../requirements2.txt"

Note: you may need to restart the kernel to use updated packages.


SI HACE FALTA WIKI-LANGCHAIN. En celdas posteriores se hacen pruebas.

In [2]:
%pip install --upgrade --quiet wikipedia langchain langchain_community


Note: you may need to restart the kernel to use updated packages.


AGENTE DE USUARIO PARA WIKIPEDIA

1) Hago una prueba pidiendole a Wiki DIABETES PRIMERO y funciona sin problema. 

2) Luego hago una prueba de idioma para ver si tb funciona en español con una búsqueda multiple de términos
queries = ["Diabetes", "Hemocromatosis", "Cáncer", "Hipertensión"]

Cambios probados:
Idioma del API de Wikipedia Y PRUEBO UNA BUSQUEDA MULTIPLE:

wiki_wiki = wikipediaapi.Wikipedia('es', headers={'User-Agent': user_agent})
Mensaje de error en español:

return "No se encontró la página para la consulta: " + query
Estos cambios aseguran que las consultas a Wikipedia se realicen en la versión en español y que cualquier mensaje de error también esté en español.

RESULTADO: Busca en la wiki en ingles, por lo que los terminos que no son iguales en los 2 idiomas, no los encuentra.

In [15]:
import wikipediaapi

# Configuración de la API de Wikipedia con agente de usuario
user_agent = 'MyApp/1.0 (example@example.com)'
wiki_wiki = wikipediaapi.Wikipedia('es', headers={'User-Agent': user_agent})

def get_wikipedia_summary(query):
    page = wiki_wiki.page(query)
    if page.exists():
        return page.summary
    else:
        return "No se encontró la página para la consulta: " + query

# Ejemplo de uso con varios términos en español
queries = ["Diabetes", "Hemocromatosis", "Cáncer", "Hipertensión"]
for query in queries:
    result = get_wikipedia_summary(query)
    print(f"Resultado para '{query}':\n{result}\n")


Resultado para 'Diabetes':
Diabetes mellitus, often known simply as diabetes, is a group of common endocrine diseases characterized by sustained high blood sugar levels. Diabetes is due to either the pancreas not producing enough insulin, or the cells of the body becoming unresponsive to the hormone's effects. Classic symptoms include thirst, polyuria, weight loss, and blurred vision. If left untreated, the disease can lead to various health complications, including disorders of the cardiovascular system, eye, kidney, and nerves. Untreated or poorly treated diabetes accounts for approximately 1.5 million deaths every year.
The major types of diabetes are type 1 and type 2. The most common treatment for type 1 is insulin replacement therapy (insulin injections), while anti-diabetic medications (such as metformin and semaglutide) and lifestyle modifications can be used to manage type 2. Gestational diabetes, a form that arises during pregnancy in some women, normally resolves shortly aft

AHORA INTEGRAMOS LANGCHAIN CON WIKI Y PROBAMOS

In [3]:
pip install sentence_transformers

Note: you may need to restart the kernel to use updated packages.


PRUEBA DE API KEY DE HUGGING FACE

Me funciona muy bien subiendo el .env de GitHub al root de aquí

In [1]:
from dotenv import load_dotenv
import os

# Cargar las variables de entorno desde el archivo .env
load_dotenv()
add_to_git_credential=True

# Obtener el token de Hugging Face desde las variables de entorno
hf_token = os.getenv("HUGGING_FACE_API_TOKEN")
if hf_token:
    from huggingface_hub import login
    login(hf_token)
else:
    print("HUGGING_FACE_API_TOKEN no encontrado en el archivo .env")

# Obtener otras variables de entorno si es necesario
groq_api_key = os.getenv("GROQ_API_KEY")
if groq_api_key:
    print("GROQ_API_KEY cargado correctamente")
else:
    print("GROQ_API_KEY no encontrado en el archivo .env")


d:\Users\casti\pinokio\bin\miniconda\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to C:\Users\casti\.cache\huggingface\token
Login successful
GROQ_API_KEY cargado correctamente


Todo sale bien, pero si se quiere quitar el aviso en rojo debe ponerse en un .sh o correr en la terminal: git config --global credential.helper store


In [1]:
from dotenv import load_dotenv
import os

# Cargar las variables de entorno desde el archivo .env
load_dotenv()

# Obtener el token de Hugging Face desde las variables de entorno
hf_token = os.getenv("HUGGING_FACE_API_TOKEN")
if hf_token:
    from huggingface_hub import login
    login(token=hf_token, add_to_git_credential=True)  # Agregar el parámetro aquí
    print("HUGGING_FACE_API_TOKEN cargado correctamente")
else:
    print("HUGGING_FACE_API_TOKEN no encontrado en el archivo .env")

# Obtener otras variables de entorno si es necesario
groq_api_key = os.getenv("GROQ_API_KEY")
if groq_api_key:
    print("GROQ_API_KEY cargado correctamente")
else:
    print("GROQ_API_KEY no encontrado en el archivo .env")


d:\Users\casti\pinokio\bin\miniconda\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Token is valid (permission: read).
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to C:\Users\casti\.cache\huggingface\token
Login successful
HUGGING_FACE_API_TOKEN cargado correctamente
GROQ_API_KEY cargado correctamente


AHORA PRUEBA DE CODIGO BUENO
Explicación
VectorStoreRetriever: Se utiliza para configurar el recuperador basado en el vectorstore.
load_qa_chain: Carga una cadena de preguntas y respuestas utilizando el PromptTemplate.
RetrievalQA: Se configura con el retriev:
D:\GitHub\TFM\Chatbot-con-FastAPI-Streamlit-y-LangChain-1\documentos_y_matcomplement\docuentreno\md\Cholesterol-Myths-vs-Facts-Spanish.md

Pruebo con un pdf pasado a .md de David En PDF

RESULTADO: LO CARGA bien, en MD.

In [2]:
CARGAR="D:\GitHub\TFM\Chatbot-con-FastAPI-Streamlit-y-LangChain-1\documentos_y_matcomplement\docuentreno\md\Cholesterol-Myths-vs-Facts-Spanish.md"

PRUEBA GENERAL CON WIKI Y LANGCHAIN: USAMOS MODELO DE HUGGING FACE GPT2.
Creamos una variable para poder cambiar el numero maximo de tokens de entrada y salida si cambiamos el modelo facilmente.

Explicación.
a) Definir el límite de tokens: Se establece MAX_INPUT_TOKENS para los tokens de entrada y MAX_NEW_TOKENS para los nuevos tokens generados.

b) Creamos una función para recortar el contexto: truncate_context recorta el contexto para asegurarse de que no se exceda el límite de tokens.

c) Crear el prompt con el contexto recortado: Se asegura de que el prompt total no exceda el límite de tokens permitido por el modelo.

### Explicación de las funciones de control de tokens

Hemos implementado funciones de control de tokens en nuestro código para asegurar que el número total de tokens (entrada más salida) no exceda el límite permitido por el modelo `gpt2`. Aquí está la explicación de las funciones clave que hemos creado y cómo puedes ajustarlas cuando cambies de modelo.

#### Función `truncate_context`

Esta función se utiliza para recortar el contexto de entrada de modo que el número total de tokens no exceda el límite permitido. Utilizamos el `GPT2Tokenizer` para contar y recortar los tokens.

**Parámetros:**
- `context`: El texto de contexto que se desea truncar.
- `max_tokens`: El número máximo de tokens permitidos para el contexto.

**Descripción:**
- La función tokeniza el contexto usando `tokenizer.encode`.
- Si el número de tokens excede `max_tokens`, se truncan los tokens al límite especificado.
- La función devuelve el contexto truncado como texto utilizando `tokenizer.decode`.

#### Definición de límites de tokens

Hemos definido los límites de tokens de entrada (`MAX_INPUT_TOKENS`) y salida (`MAX_NEW_TOKENS`) para asegurarnos de que el número total de tokens no exceda 1024, el límite para `gpt2`.
**Descripción:**
- `MAX_INPUT_TOKENS`: El número máximo de tokens permitidos para el contexto de entrada.
- `MAX_NEW_TOKENS`: El número máximo de tokens permitidos para la respuesta generada por el modelo.

#### Aplicación en el código

Utilizamos la función `truncate_context` para recortar el contexto antes de crear el prompt y enviar la solicitud al modelo.

### Ajustes para cambiar de modelo

Cuando se cambie de modelo, necesitaremos ajustar los límites de tokens (`MAX_INPUT_TOKENS` y `MAX_NEW_TOKENS`) según las especificaciones del nuevo modelo. La mayoría de los modelos tienen documentación que indica el límite máximo de tokens que pueden manejar. Por ejemplo, si cambia a un modelo con un límite de 2048 tokens, podrías ajustar los límites de la siguiente manera:

```python
MAX_INPUT_TOKENS = 1948  # Ajustar según el nuevo límite
MAX_NEW_TOKENS = 100
```

Asegurarse de revisar la documentación del nuevo modelo para conocer el límite exacto de tokens y ajustar las variables en consecuencia.


In [1]:
import wikipediaapi
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import HuggingFaceHub
from dotenv import load_dotenv
import os
import requests
from transformers import GPT2Tokenizer

# Cargar las variables de entorno desde el archivo .env
load_dotenv()

# Usar root como base del GitHub
root = "https://github.com/davidfdezmartin/Chatbot-con-FastAPI-Streamlit-y-LangChain/blob/main"

# Verificar y obtener el token de Hugging Face desde las variables de entorno
hf_token = os.getenv("HUGGING_FACE_API_TOKEN")
if not hf_token:
    raise ValueError("HUGGING_FACE_API_TOKEN no encontrado en el archivo .env")

# Configuración de la API de Wikipedia con agente de usuario
user_agent = 'MyApp/1.0 (example@example.com)'
wiki_wiki = wikipediaapi.Wikipedia('es', headers={'User-Agent': user_agent})

def get_wikipedia_summary(query):
    page = wiki_wiki.page(query)
    if page.exists():
        return page.summary
    else:
        return "No se encontró la página para la consulta: " + query

# Ejemplo de uso con varios términos en español
queries = ["Diabetes", "Hemocromatosis", "Cáncer", "Hipertensión"]
for query in queries:
    result = get_wikipedia_summary(query)
    print(f"Resultado para '{query}':\n{result}\n")

# URL del archivo en GitHub
file_url = f"{root}/documentos_y_matcomplement/docuentreno/md/Cholesterol-Myths-vs-Facts-Spanish.md?raw=true"

# Descargar el contenido del archivo
response = requests.get(file_url)
if response.status_code != 200:
    raise RuntimeError(f"Error al descargar el archivo desde {file_url}")

# Guardar el contenido en un archivo temporal
with open("temp.md", "w", encoding="utf-8") as temp_file:
    temp_file.write(response.text)

# Cargar el documento .md desde el archivo temporal
text_loader = TextLoader("temp.md")
documents = text_loader.load()

# Dividir texto
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)  # Reducir el tamaño del fragmento
texts = text_splitter.split_documents(documents)

# Crear embeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectorstore = FAISS.from_documents(texts, embeddings)

# Crear el prompt con variables de entrada especificadas
prompt_template = PromptTemplate(
    template="Responde la pregunta basándote en el siguiente contexto: {context}\nPregunta: {question}",
    input_variables=["context", "question"]
)

# Definir el límite de tokens
MAX_INPUT_TOKENS = 824  # Ajustar según sea necesario
MAX_NEW_TOKENS = 100
TOTAL_TOKENS = MAX_INPUT_TOKENS + MAX_NEW_TOKENS

# Inicializar el tokenizer de GPT-2
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

# Función para recortar el contexto si es necesario
def truncate_context(context, max_tokens):
    tokens = tokenizer.encode(context)
    if len(tokens) > max_tokens:
        tokens = tokens[:max_tokens]
        return tokenizer.decode(tokens, clean_up_tokenization_spaces=True)
    return context

# Crear la cadena de preguntas y respuestas
llm = HuggingFaceHub(repo_id="gpt2", model_kwargs={"temperature": 0.7}, huggingfacehub_api_token=hf_token)
qa_chain = load_qa_chain(llm, chain_type="stuff")

# Crear el retrieval QA chain utilizando el vectorstore y la cadena de QA
retrieval_qa = RetrievalQA(
    retriever=vectorstore.as_retriever(),
    combine_documents_chain=qa_chain
)

# Ejemplo de uso de la cadena de preguntas y respuestas con control de longitud de tokens
question = "¿Cuál es el tema principal del documento?"

# Obtener el contexto
context = " ".join([doc.page_content for doc in texts])
context = truncate_context(context, MAX_INPUT_TOKENS)

# Crear el prompt con el contexto recortado
prompt = prompt_template.format(context=context, question=question)

# Generar la respuesta utilizando el modelo de lenguaje
result = llm(prompt, max_new_tokens=MAX_NEW_TOKENS)

print("QA Result:\n", result)


d:\Users\casti\pinokio\bin\miniconda\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Resultado para 'Diabetes':
Diabetes mellitus, often known simply as diabetes, is a group of common endocrine diseases characterized by sustained high blood sugar levels. Diabetes is due to either the pancreas not producing enough insulin, or the cells of the body becoming unresponsive to the hormone's effects. Classic symptoms include thirst, polyuria, weight loss, and blurred vision. If left untreated, the disease can lead to various health complications, including disorders of the cardiovascular system, eye, kidney, and nerves. Untreated or poorly treated diabetes accounts for approximately 1.5 million deaths every year.
The major types of diabetes are type 1 and type 2. The most common treatment for type 1 is insulin replacement therapy (insulin injections), while anti-diabetic medications (such as metformin and semaglutide) and lifestyle modifications can be used to manage type 2. Gestational diabetes, a form that arises during pregnancy in some women, normally resolves shortly aft

d:\Users\casti\pinokio\bin\miniconda\lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\casti\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
d:\Users\casti\pinokio\bin\miniconda\lib\site-packages\huggingface_hub\file_download.py:1

QA Result:
 Responde la pregunta basándote en el siguiente contexto: ﻿<a name="br1"></a> 

COLESTEROL: MITOS Y VERDADES

Mito: Sólo necesitará analizar sus niveles de colesterol cuando tenga más de 30 años.

Verdad: La Asociación Americana del Corazón (AHA) recomienda analizar los niveles de colesterol

una vez entre los 9 y 11 años, y de nuevo entre los 17 y 21 años para niños y adultos jóvenes sin otros

factores de riesgos o un historial familiar de enfermedad cardíaca temprana. Después de los 20 factores de riesgos o un historial familiar de enfermedad cardíaca temprana. Después de los 20

años, su doctor volverá a analizar su colesterol y otros factores de riesgo cada cuatro a seis años

siempre y cuando su riesgo se mantenga bajo.

Mito: Sólo la gente con sobrepeso u obesidad tiene colesterol alto.

Verdad: Las personas con cualquier tipo de cuerpo pueden tener colesterol alto. Tener sobrepeso Verdad: Las personas con cualquier tipo de cuerpo pueden tener colesterol alto. Tener s

In [ ]:
## Read the ppdfs from the folder
loader=PyPDFDirectoryLoader(f,{root}/"documentos_y_matcomplement/docuentreno/pdf")

documents=loader.load()

text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)

final_documents=text_splitter.split_documents(documents)
final_documents[0]

MIERCOLES 29 MAYO.
VOY REVISANDO POR AQUÍ HASTA AHORA CARGA DE MD Y LLM DE HUGGING FACES PERFECTO Y WIKIPEDIA PERFECTA

Hoy 29/05 se ha revisado para que cargue los documentos a partir del root de github para hacer el codigo portable y no dependiente de la copia local del github particular.



Hago una prueba del texto que responde y su formato

In [ ]:
import requests
from langchain_community.document_loaders import TextLoader

# Definir root como la URL base de GitHub raw content
root = "https://raw.githubusercontent.com/davidfdezmartin/Chatbot-con-FastAPI-Streamlit-y-LangChain/main"

print("Root de GitHub:", root)

# URL del archivo en GitHub
file_path = f"{root}/documentos_y_matcomplement/docuentreno/md/Cholesterol-Myths-vs-Facts-Spanish.md"
print("URL del archivo:", file_path)

# Descargar el contenido del archivo
response = requests.get(file_path)
if response.status_code != 200:
    raise RuntimeError(f"Error al descargar el archivo desde {file_path}")

# Guardar el contenido en un archivo temporal
with open("temp.md", "w", encoding="utf-8") as temp_file:
    temp_file.write(response.text)

# Cargar el documento .md desde el archivo temporal
text_loader = TextLoader("temp.md")
documents = text_loader.load()
print("Contenido del documento:", documents)


Root de GitHub: https://raw.githubusercontent.com/davidfdezmartin/Chatbot-con-FastAPI-Streamlit-y-LangChain/main
URL del archivo: https://raw.githubusercontent.com/davidfdezmartin/Chatbot-con-FastAPI-Streamlit-y-LangChain/main/documentos_y_matcomplement/docuentreno/md/Cholesterol-Myths-vs-Facts-Spanish.md
Contenido del documento: [Document(page_content='\ufeff<a name="br1"></a> \n\nCOLESTEROL: MITOS Y VERDADES\n\nMito: Sólo necesitará analizar sus niveles de colesterol cuando tenga más de 30 años.\n\nVerdad: La Asociación Americana del Corazón (AHA) recomienda analizar los niveles de colesterol\n\nuna vez entre los 9 y 11 años, y de nuevo entre los 17 y 21 años para niños y adultos jóvenes sin otros\n\nfactores de riesgos o un historial familiar de enfermedad cardíaca temprana. Después de los 20\n\naños, su doctor volverá a analizar su colesterol y otros factores de riesgo cada cuatro a seis años\n\nsiempre y cuando su riesgo se mantenga bajo.\n\nMito: Sólo la gente con sobrepeso u obe

JUEVES 29/MAYO
ESTE CODIGO CLON DEL ANTERIOR USAN .ENV Y ROOT PARA ACORTAR DONDE APUNTAN LOS PROGRAMAS

Explicación de las mejoras
Uso de os.getenv("ROOT") para obtener la URL base del repositorio desde el archivo .env.
Construcción de la URL del archivo con os.path.join para asegurar compatibilidad con diferentes sistemas operativos.

Este enfoque mantiene el código limpio y permite a cualquiera usar el notebook sin depender de rutas específicas del sistema local.

In [ ]:
import wikipediaapi
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import HuggingFaceHub
from dotenv import load_dotenv
import os
import requests
from transformers import GPT2Tokenizer

# Cargar las variables de entorno desde el archivo .env
load_dotenv()

# Usar root como base del GitHub
root = os.getenv("ROOT")
if not root:
    raise ValueError("ROOT no encontrado en el archivo .env")

print("Root de GitHub:", root)

# Verificar y obtener el token de Hugging Face desde las variables de entorno
hf_token = os.getenv("HUGGING_FACE_API_TOKEN")
if not hf_token:
    raise ValueError("HUGGING_FACE_API_TOKEN no encontrado en el archivo .env")

# Configuración de la API de Wikipedia con agente de usuario
user_agent = 'MyApp/1.0 (example@example.com)'
wiki_wiki = wikipediaapi.Wikipedia('es', headers={'User-Agent': user_agent})

def get_wikipedia_summary(query):
    page = wiki_wiki.page(query)
    if page.exists():
        return page.summary
    else:
        return "No se encontró la página para la consulta: " + query

# Ejemplo de uso con varios términos en español
queries = ["Diabetes", "Hemocromatosis", "Cáncer", "Hipertensión"]
for query in queries:
    result = get_wikipedia_summary(query)
    print(f"Resultado para '{query}':\n{result}\n")
    
    # PRUEBO METER LA VARIABLE ROOT A MANITA SIN EL .env y lo imprimo para verlo
    
# URL del archivo en GitHub
file_path = f"{root}/documentos_y_matcomplement/docuentreno/md/Cholesterol-Myths-vs-Facts-Spanish.md"
print("URL del archivo:", file_path)

# Descargar el contenido del archivo
response = requests.get(file_path)
if response.status_code != 200:
    raise RuntimeError(f"Error al descargar el archivo desde {file_path}")

# Guardar el contenido en un archivo temporal
with open("temp.md", "w", encoding="utf-8") as temp_file:
    temp_file.write(response.text)

# Cargar el documento .md desde el archivo temporal
text_loader = TextLoader("temp.md")
documents = text_loader.load()
print("Contenido del documento:", documents)



# Dividir texto
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)  # Reducir el tamaño del fragmento
texts = text_splitter.split_documents(documents)

# Crear embeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectorstore = FAISS.from_documents(texts, embeddings)

# Crear el prompt con variables de entrada especificadas
prompt_template = PromptTemplate(
    template="Responde la pregunta basándote en el siguiente contexto: {context}\nPregunta: {question}",
    input_variables=["context", "question"]
)

# Definir el límite de tokens
MAX_INPUT_TOKENS = 824  # Ajustar según sea necesario
MAX_NEW_TOKENS = 100
TOTAL_TOKENS = MAX_INPUT_TOKENS + MAX_NEW_TOKENS

# Inicializar el tokenizer de GPT-2
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

# Función para recortar el contexto si es necesario
def truncate_context(context, max_tokens):
    tokens = tokenizer.encode(context)
    if len(tokens) > max_tokens:
        tokens = tokens[:max_tokens]
        return tokenizer.decode(tokens, clean_up_tokenization_spaces=True)
    return context

# Crear la cadena de preguntas y respuestas
llm = HuggingFaceHub(repo_id="gpt2", model_kwargs={"temperature": 0.7}, huggingfacehub_api_token=hf_token)
qa_chain = load_qa_chain(llm, chain_type="stuff")

# Crear el retrieval QA chain utilizando el vectorstore y la cadena de QA
retrieval_qa = RetrievalQA(
    retriever=vectorstore.as_retriever(),
    combine_documents_chain=qa_chain
)

# Ejemplo de uso de la cadena de preguntas y respuestas con control de longitud de tokens
question = "¿Cuál es el tema principal del documento?"

# Obtener el contexto
context = " ".join([doc.page_content for doc in texts])
context = truncate_context(context, MAX_INPUT_TOKENS)

# Crear el prompt con el contexto recortado
prompt = prompt_template.format(context=context, question=question)

# Generar la respuesta utilizando el modelo de lenguaje
result = llm(prompt, max_new_tokens=MAX_NEW_TOKENS)

print("QA Result:\n", result)


Root de GitHub: https://raw.githubusercontent.com/davidfdezmartin/Chatbot-con-FastAPI-Streamlit-y-LangChain/main
Resultado para 'Diabetes':
Diabetes mellitus, often known simply as diabetes, is a group of common endocrine diseases characterized by sustained high blood sugar levels. Diabetes is due to either the pancreas not producing enough insulin, or the cells of the body becoming unresponsive to the hormone's effects. Classic symptoms include thirst, polyuria, weight loss, and blurred vision. If left untreated, the disease can lead to various health complications, including disorders of the cardiovascular system, eye, kidney, and nerves. Untreated or poorly treated diabetes accounts for approximately 1.5 million deaths every year.
The major types of diabetes are type 1 and type 2. The most common treatment for type 1 is insulin replacement therapy (insulin injections), while anti-diabetic medications (such as metformin and semaglutide) and lifestyle modifications can be used to mana

d:\Users\casti\pinokio\bin\miniconda\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Token indices sequence length is longer than the specified maximum sequence length for this model (2310 > 1024). Running this sequence through the model will result in indexing errors


QA Result:
 Responde la pregunta basándote en el siguiente contexto: ﻿<a name="br1"></a> 

COLESTEROL: MITOS Y VERDADES

Mito: Sólo necesitará analizar sus niveles de colesterol cuando tenga más de 30 años.

Verdad: La Asociación Americana del Corazón (AHA) recomienda analizar los niveles de colesterol

una vez entre los 9 y 11 años, y de nuevo entre los 17 y 21 años para niños y adultos jóvenes sin otros

factores de riesgos o un historial familiar de enfermedad cardíaca temprana. Después de los 20 factores de riesgos o un historial familiar de enfermedad cardíaca temprana. Después de los 20

años, su doctor volverá a analizar su colesterol y otros factores de riesgo cada cuatro a seis años

siempre y cuando su riesgo se mantenga bajo.

Mito: Sólo la gente con sobrepeso u obesidad tiene colesterol alto.

Verdad: Las personas con cualquier tipo de cuerpo pueden tener colesterol alto. Tener sobrepeso Verdad: Las personas con cualquier tipo de cuerpo pueden tener colesterol alto. Tener s

# Pasos adicionales que acabo de  seguir:

Eliminar caracteres especiales no deseados.
Asegurarte de que el contexto no se corte a mitad de una frase o sección importante.

## Explicación
**Función clean_text**: Se crea una función para eliminar caracteres especiales no deseados y limpiar el contenido del documento.
Aplicar limpieza al contenido: Se aplica la función clean_text a cada documento cargado para asegurar que el contenido sea claro y esté bien formateado.
Imprimir el contenido limpio: Se imprime el contenido limpio del documento para verificar el resultado.

## LO ACABO DE SUBIR BIEN EXPLICADO EN MD AL GITHUB
https://raw.githubusercontent.com/davidfdezmartin/Chatbot-con-FastAPI-Streamlit-y-LangChain/main/documentos_y_matcomplement/Presentacion/29%2005%20tfm%20limpiar%20salida%20y%20caragr%20de%20github.md

In [ ]:
import requests
from langchain_community.document_loaders import TextLoader
from dotenv import load_dotenv
import os

# Cargar las variables de entorno desde el archivo .env
load_dotenv()

# Usar root como base del GitHub
root = os.getenv("ROOT")
if not root:
    raise ValueError("ROOT no encontrado en el archivo .env")

print("Root de GitHub:", root)

# URL del archivo en GitHub
file_path = f"{root}/documentos_y_matcomplement/docuentreno/md/Cholesterol-Myths-vs-Facts-Spanish.md"
print("URL del archivo:", file_path)

# Descargar el contenido del archivo
response = requests.get(file_path)
if response.status_code != 200:
    raise RuntimeError(f"Error al descargar el archivo desde {file_path}")

# Guardar el contenido en un archivo temporal
with open("temp.md", "w", encoding="utf-8") as temp_file:
    temp_file.write(response.text)

# Cargar el documento .md desde el archivo temporal
text_loader = TextLoader("temp.md")
documents = text_loader.load()

# Limpiar el contenido del documento
def clean_text(text):
    # Eliminar caracteres especiales no deseados
    text = text.replace('\ufeff', '')
    text = text.replace('<a name="br1"></a>', '')
    return text.strip()

# Procesar y limpiar el contenido del documento
cleaned_documents = []
for doc in documents:
    cleaned_text = clean_text(doc.page_content)
    cleaned_documents.append(cleaned_text)

# Mostrar el contenido limpio del documento
for doc in cleaned_documents:
    print("Contenido limpio del documento:", doc)


Root de GitHub: https://raw.githubusercontent.com/davidfdezmartin/Chatbot-con-FastAPI-Streamlit-y-LangChain/main
URL del archivo: https://raw.githubusercontent.com/davidfdezmartin/Chatbot-con-FastAPI-Streamlit-y-LangChain/main/documentos_y_matcomplement/docuentreno/md/Cholesterol-Myths-vs-Facts-Spanish.md
Contenido limpio del documento: COLESTEROL: MITOS Y VERDADES

Mito: Sólo necesitará analizar sus niveles de colesterol cuando tenga más de 30 años.

Verdad: La Asociación Americana del Corazón (AHA) recomienda analizar los niveles de colesterol

una vez entre los 9 y 11 años, y de nuevo entre los 17 y 21 años para niños y adultos jóvenes sin otros

factores de riesgos o un historial familiar de enfermedad cardíaca temprana. Después de los 20

años, su doctor volverá a analizar su colesterol y otros factores de riesgo cada cuatro a seis años

siempre y cuando su riesgo se mantenga bajo.

Mito: Sólo la gente con sobrepeso u obesidad tiene colesterol alto.

Verdad: Las personas con cualq

COMPRUEBO LA LONGITUD DEL DOCUMENTO CREADO

In [ ]:
len(doc)

5454

Aquí haciamos una prueba con un vectorizador de Hugging Face un modelo pequeño: BAAI/bge-small-en-v1.5

In [10]:
## Embedding Using Huggingface
huggingface_embeddings=HuggingFaceBgeEmbeddings(
    model_name="BAAI/bge-small-en-v1.5",      #sentence-transformers/all-MiniLM-l6-v2
    model_kwargs={'device':'cpu'},
    encode_kwargs={'normalize_embeddings':True}

)

NameError: name 'HuggingFaceBgeEmbeddings' is not defined

In [9]:
import  numpy as np
print(np.array(huggingface_embeddings.embed_query(final_documents[0].page_content)))
print(np.array(huggingface_embeddings.embed_query(final_documents[0].page_content)).shape)

NameError: name 'huggingface_embeddings' is not defined

In [11]:
## VectorStore Creation
vectorstore=FAISS.from_documents(final_documents[:120],huggingface_embeddings)

NameError: name 'final_documents' is not defined

In [ ]:
## Query using Similarity Search
query="WHAT IS HEALTH INSURANCE COVERAGE?"
relevant_docments=vectorstore.similarity_search(query)

print(relevant_docments[0].page_content)

2 U.S. Census Bureau
WHAT IS HEALTH INSURANCE COVERAGE?
This brief presents state-level estimates of health insurance coverage 
using data from the American Community Survey (ACS). The  
U.S. Census Bureau conducts the ACS throughout the year; the 
survey asks respondents to report their coverage at the time of 
interview. The resulting measure of health insurance coverage, 
therefore, reflects an annual average of current comprehensive 
health insurance coverage status.* This uninsured rate measures a 
different concept than the measure based on the Current Population 
Survey Annual Social and Economic Supplement (CPS ASEC). 
For reporting purposes, the ACS broadly classifies health insurance 
coverage as private insurance or public insurance. The ACS defines 
private health insurance as a plan provided through an employer 
or a union, coverage purchased directly by an individual from an 
insurance company or through an exchange (such as healthcare.


In [ ]:
retriever=vectorstore.as_retriever(search_type="similarity",search_kwargs={"k":3})
print(retriever)

tags=['FAISS', 'HuggingFaceBgeEmbeddings'] vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x00000159F4860C10> search_kwargs={'k': 3}


In [ ]:
import os
os.environ['HUGGINGFACEHUB_API_TOKEN']=""

The Hugging Face Hub is an platform with over 350k models, 75k datasets, and 150k demo apps (Spaces), all open source and publicly available, in an online platform where people can easily collaborate and build ML together.

In [ ]:
from langchain_community.llms import HuggingFaceHub

hf=HuggingFaceHub(
    repo_id="mistralai/Mistral-7B-v0.1",
    model_kwargs={"temperature":0.1,"max_length":500}

)
query="What is the health insurance coverage?"
hf.invoke(query)

e:\Langchain\venv\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.llms.huggingface_hub.HuggingFaceHub` was deprecated in langchain-community 0.0.21 and will be removed in 0.2.0. Use HuggingFaceEndpoint instead.
  warn_deprecated(


'What is the health insurance coverage?\n\nThe health insurance coverage is a contract between the insurer and the insured. The insurer agrees to pay the insured for the medical expenses incurred by the insured. The insured agrees to pay the premium to the insurer.\n\nWhat is the health insurance coverage?\n\nThe health insurance coverage is a contract between the insurer and the insured. The insurer agrees to pay the insured for the medical expenses incurred by the insured.'

In [ ]:
#Hugging Face models can be run locally through the HuggingFacePipeline class.
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline

hf = HuggingFacePipeline.from_model_id(
    model_id="mistralai/Mistral-7B-v0.1",
    task="text-generation",
    pipeline_kwargs={"temperature": 0, "max_new_tokens": 300}
)

llm = hf
llm.invoke(query)

In [ ]:
prompt_template="""
Use the following piece of context to answer the question asked.
Please try to provide the answer only based on the context

{context}
Question:{question}

Helpful Answers:
 """

In [ ]:
prompt=PromptTemplate(template=prompt_template,input_variables=["context","question"])

In [ ]:
retrievalQA=RetrievalQA.from_chain_type(
    llm=hf,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt":prompt}
)

In [ ]:
query="""DIFFERENCES IN THE
UNINSURED RATE BY STATE
IN 2022"""

In [ ]:
# Call the QA chain with our query.
result = retrievalQA.invoke({"query": query})
print(result['result'])


Use the following piece of context to answer the question asked.
Please try to provide the answer only based on the context

comparison of ACS and CPS ASEC measures 
of health insurance coverage, refer to < www.
census.gov/topics/health/health-insurance/
guidance.html >.
9 Respondents may have more than one 
health insurance coverage type at the time 
of interview. As a result, adding the total 
number of people with private coverage and 
the total number with public coverage will 
sum to more than the total number with any 
coverage.• From 2021 to 2022, nine states 
reported increases in private 
coverage, while seven reported 
decreases (Appendix Table B-2). 
DIFFERENCES IN THE 
UNINSURED RATE BY STATE 
IN 2022
In 2022, uninsured rates at the 
time of interview ranged across 
states from a low of 2.4 percent 
in Massachusetts to a high of 16.6 
percent in Texas, compared to the 
national rate of 8.0 percent.10 Ten 
of the 15 states with uninsured 
10 The uninsured rates in the Distr